### 목표
- 머신러닝의 과정에 대해서 정리해보자
- 실제 데이터를 통해서 고객에게 서비스 할 제품에 대해 근거가 될만한 자료를 생성해보자

#### 1. 문제 정의
- 모델을 구축하는 목표 설정
- 모델 선택 내용 정의
- 분류 vs 회귀 정의
- 어떤 데이터가 필요할까?
-------
> **시나리오** 🌏 (서비스기획자)
 
```
- A업체는 가정 내 온습도를 측정하는 기존 제품을 통해 고객에게 서비스를 제공해 왔음
- 한때 업계 선두를 유지했으나, 다양한 경쟁사의 등장으로 인해 최근 판매량이 감소하고 있는 상황임
- 단순히 제품 판매에 의존하는 비즈니스 모델로는 지속적인 성장이 어렵다고 판단하여, 새로운 고객 가치를 창출할 수 있는 제품 개발을 추진하게 됨
- 온습도 센서로부터 수집한 데이터를 기반으로 AI 제습기를 개발하여, 고객의 편의성을 높이고 경쟁사 대비 우위를 선점하려고 함
```

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### 2. 데이터 수집
- 데이터 정리
- 웹 크롤링
- API
- csv 파일
- DB
- 설문조사

In [4]:
# 출처 : kaggle
data  = pd.read_csv('./data/온습도 관측 데이터.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1546 entries, 0 to 1545
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   T            1546 non-null   float64
 1   RH           1546 non-null   float64
 2   AH           1546 non-null   float64
 3   Comfortable  1546 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 48.4 KB


In [6]:
data.head()

,T,RH,AH,Comfortable
0,21.025,30.625000,0.753814,0
1,9.250,37.550000,0.439072,1
2,35.825,28.724999,1.662621,0
3,15.975,35.824999,0.645597,1
4,12.200,69.575001,0.985989,0


- 행 1546, 열 4
- 결측치 x
- 데이터 타입 : 수치형
- Comfortable : 분류

> **데이터 설명**
 
* 3개의 센서 데이터가 수집됨 (원활한 실습을 위해 라벨은 미리 달아두었음)
* 데이터 명세
 
| Column Name | 컬럼명 | 설명 |
|-------------|--------|------|
| T | 온도 | 주변 환경의 온도 (섭씨) |
| RH | 상대습도 | 공기 중 습도의 비율 (%) |
| AH | 절대습도 | 공기 중 실제 수증기량 |
| Comfortable | 쾌적 여부 | 환경이 쾌적한지 나타내는 이진값 - 0(불쾌), 1(상쾌) |

#### 3. 데이터 전처리(preprocessing)
- 결측치 제거 및 대체
- 이상치 제거 및 대체
- 변수(컬럼, 특성, feature) 처리 : 특성추가, 삭제, 선택, 스케일링(단위 조정), 인코딩
------

#### 4. EDA(탐색적 데이터 분석)
- 데이터의 기술통계량(최대,최소,사분위수,평균,중앙 등), 시각화 등을 통한 다양한 각도에서 데이터를 관찰하는 단계
- 데이터 전처리를 하게되는 근거로 삼을수 있음

In [20]:
# 기술통계량
data.describe().T

,count,mean,std,min,25%,50%,75%,max
T,1546.0,18.702808,8.787124,-1.900000,11.825000,19.225000,25.168750,44.600000
RH,1546.0,43.917987,14.464104,9.225000,36.150000,38.724999,51.468750,88.725000
AH,1546.0,0.977192,0.423748,0.198757,0.639129,0.959642,1.289465,2.139496
Comfortable,1546.0,0.482536,0.499857,0.000000,0.000000,0.000000,1.000000,1.000000


In [28]:
# 불쾌/상쾌를 판단하기 위한 가장 중요한 특성(T,RH,AH)은 무엇일까?
# -- 여러 특성간의 관계를 파악해보자!
# corr : 상관계수 값을 통해서 특성간의 연관성을 파악하자(상관관계)
# 특성간의 방향성을 파악해서 비례, 반비례, 의미x
# -1 ~ 1 범위로 수치 표현
# 0 의미가 없음
data.corr()['Comfortable']

T              0.080162
RH            -0.431584
AH            -0.113466
Comfortable    1.000000
Name: Comfortable, dtype: float64

#### 상관계수 해석
 
```
|r| = 절대값  
 
· 0.0 <= |r| < 0.2 : 상관관계가 없다. = 선형의 관계가 없다.  
 
· 0.2 <= |r| < 0.4 : 약한 상관관계가 있다.  
 
· 0.4 <= |r| < 0.6 : 보통의 상관관계가 있다.  
 
· 0.6 <= |r| < 0.8 : 강한 (높은) 상관관계가 있다.  
 
· 0.8 <= |r| <= 1.0 : 매우 강한 (매우 높은) 상관관계가 있다.
```

- RH : 상대습도가 상대적으로 상관관계가 높음! (약한 상관관계)

In [33]:
# 데이터 분리 진행하기
# 문제, 답 분리
# 문제 : 특성, 독립변수, 설명변수, feature, X
# 답 : label, 레이블, 라벨, 종속변수, 반응변수, y
# * 왜 X 대문자인가? 문제데이터 컬럼은 여러개(차원 개수 많음 > 대문자로 표현)
data.head()

,T,RH,AH,Comfortable
0,21.025,30.625000,0.753814,0
1,9.250,37.550000,0.439072,1
2,35.825,28.724999,1.662621,0
3,15.975,35.824999,0.645597,1
4,12.200,69.575001,0.985989,0


In [41]:
X = data.iloc[:,:3]   # data.drop('Comfortable', axis=1)
y = data['Comfortable']

print(X.shape, y.shape)

(1546, 3) (1546,)


In [45]:
# train, test 분리
# 행분리
# -- 직접 비율에 따른 개수를 파악해서 분리해보자!
X.shape[0]*0.7   # 1082개 train, 나머지 test

1082.1999999999998

In [49]:
X_train = X[:1082]
X_test = X[1082:]
y_tain = y[:1082]
y_test = y[1082:]

X_train.shape, X_test.shape, y_tain.shape, y_test.shape

((1082, 3), (464, 3), (1082,), (464,))

- 만약, 0은 0대로, 1은 1대로 정리된 데이터라면 위 방식이 괜찮을까?
- -- 랜덤하게 섞는 기능 + 비율에 따라 분리 :  sklearn 패키지 내부에 있는 train_test_split()

In [55]:
# 데이터 분할하는 도구 사용하기
from sklearn.model_selection import train_test_split

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=7) # 7규칙으로 랜덤 섞어줌

print('훈련용', X_train.shape, y_train.shape)
print('테스트용', X_test.shape, y_test.shape)
# 랜덤하게 행을 섞어서 분리해줌! 데이터의 다양성을 표현해주기 위해서(0,1 제대로 학습할 수 잇게)

훈련용 (1082, 3) (1082,)
테스트용 (464, 3) (464,)


#### 5. 머신러닝 모델 선택
- 이진분류
- 분류 모델 선택
- 종류는 다양하지만, knn모델을 사용해보자!
- 하이퍼 파라미터를 조정해보자(튜닝)

In [81]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [88]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)
pred = knn_model.predict(X_test)
accuracy_score(y_test, pred)

0.9870689655172413

In [85]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
pred = knn_model.predict(X_test)
accuracy_score(y_test, pred)

0.9762931034482759

- n_neighbors 5보다 3이 더 정확도가 높다

In [90]:
knn_model = KNeighborsClassifier(n_neighbors=7)
knn_model.fit(X_train, y_train)
pred = knn_model.predict(X_test)
accuracy_score(y_test, pred)

0.9698275862068966

In [94]:
knn_model = KNeighborsClassifier(n_neighbors=4)
knn_model.fit(X_train, y_train)
pred = knn_model.predict(X_test)
accuracy_score(y_test, pred)
# 이웃의 수를 다수결 판단 알고리즘 : 2:2 / 4:4 *거리값이 가까운 데이터의 중요도를 높여서 예측 결과를 진단
# 대체적으로 n_neighbors은 홀수 채용

0.9892241379310345

- test 1번만 진행했기 때문에, k가 4로 설정한 것이 모델의 성능이 최고로 높다라고 단정짓기에는 어려움
- 여러번 판단해볼 필요가 있고 train의 성능도 좋고 test성능도 좋은 형태의 모델을 찾는 노력 필요

- 일반화 (Generalization): 훈련 데이터뿐만 아니라 새로운 데이터에도 잘 작동하는 능력.
- 과대적합 (Overfitting): 훈련 데이터에 너무 맞춰져서 새로운 데이터에 대한 성능이 나쁜 상태.
- 과소적합 (Underfitting): 모델이 너무 단순하여 훈련 데이터와 새로운 데이터 모두에서 성능이 낮은 상태.

#### 모델의 목표는 일반화를 잘 하도록 하여, 과대적합과 과소적합을 피하는 것

In [102]:
# 8. 모델 활용(서비스화)
# - 웹, 앱 서비스화
# - 데이터 분석에 대한 근거를 수립할 때 활용(보고서)